In [2]:
import random
import operator
import numpy as np
import matplotlib.pyplot as plt
from deap import creator, base, tools, algorithms

# Données des machines de nettoyage (nom, temps de fonctionnement en heures, productivité en m²)
scrubbers = [("A1", 1, 1225), ("A2", 1.5, 1575), ("A3", 2.5, 2475), ("A4", 2.5, 1750), 
             ("A5", 3, 1750), ("A6", 3.5, 3150), ("A7", 2.5, 2700), ("A8", 2.5, 3150), 
             ("A9", 3.5, 3150), ("A10", 4, 3825), ("B1", 1.5, 1400), ("B2", 2.5, 1720), 
             ("B3", 3.5, 1720), ("B4", 3.5, 2200), ("B5", 5, 2200), ("C1", 8, 5600), 
             ("C2", 3, 2970), ("C3", 3.5, 2460), ("C4", 4.5, 7740), ("C5", 5, 9000)]

# Données des sites de nettoyage (nom, surface de nettoyage en m², temps de nettoyage en heures)
sites = [("Site 1", 23900, 7), ("Site 2", 19200, 4), ("Site 3", 16400, 6), ("Site 4", 16200, 6), 
         ("Site 5", 33000, 3)]


# Fonction d'évaluation d'un individu
def evaluate(individual):
    #
    #Évalue la qualité d'un individu en calculant le temps total de nettoyage de tous les sites avec les machines 
    #attribuées selon cet individu.
    #
    # Création d'un dictionnaire qui contiendra la liste des machines attribuées à chaque site
    site_assignments = {site[0]: [] for site in sites}
    
    # Attribution des machines aux sites
    for machine, site_index in zip(individual, range(len(sites))):
        site_assignments[sites[site_index][0]].append(scrubbers[machine][0])
    
    # Calcul du temps total de nettoyage pour chaque site
    total_cleaning_time = 0
    for site in sites:
        cleaning_time = site[1] / sum(scrubbers[machine][2] for machine in site_assignments[site[0]])
        total_cleaning_time += cleaning_time if cleaning_time > site[2] else site[2]
    
    return total_cleaning_time,

# Initialisation de la classe Fitness pour maximiser la qualité des individus
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# Initialisation de la classe individu
creator.create("Individual", list, fitness=creator.FitnessMax)
               
class CleaningProblem:
    def __init__(self, scrubbers, sites):
        
        self.scrubbers = scrubbers
        
        # Création des listes de poids et de bornes
        self.weights = np.array([scrubber[1] for scrubber in scrubbers])
        self.bounds = [(0, site[1] / scrubbers[-1][2]) for site in sites]
        
        self.sites = sites
        self.num_scrubbers = len(scrubbers)
        self.num_sites = len(sites)

        
    
    def eval_solution(self, individual):
        # Calcul de la durée totale de nettoyage
        cleaning_times = [sum(self.weights[np.where(individual == i)[0]]) for i in range(self.num_sites)]
        total_time = max(cleaning_times)

        # Calcul de la distance parcourue par chaque machine de nettoyage
        distances = [0] * self.num_scrubbers
        for i in range(1, self.num_sites):
            for j in range(self.num_scrubbers):
                if individual[i-1] == j:
                    distances[j] += self.sites[i][1]

        # Calcul du coût total de la solution
        total_cost = sum([distance * scrubber[2] for distance, scrubber in zip(distances, self.scrubbers)])

        return total_time, total_cost

problem = CleaningProblem(scrubbers, sites)

POPULATION_SIZE = 400
P_CROSSOVER = 1
P_MUTATION = 0.05
MAX_GENERATIONS = 300
HALL_OF_FAME_SIZE = 10
RANDOM_SEED = 50
TOURNAMENT_SIZE = 3

random.seed(RANDOM_SEED)

creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)
toolbox = base.Toolbox()

toolbox.register("indexes", random.sample, range(problem.num_scrubbers), problem.num_sites)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indexes)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", problem.eval_solution)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=1.0/problem.num_sites)
toolbox.register("select", tools.selTournament, tournsize=TOURNAMENT_SIZE)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("min", lambda x: (x[0], x[1]))

hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

population = toolbox.population(n=POPULATION_SIZE)
population, logbook = algorithms.eaSimple(population, toolbox, cxpb = P_CROSSOVER, mutpb = P_MUTATION, ngen=MAX_GENERATIONS, stats=stats, halloffame=hof, verbose=True)
# Création de la population initiale
population = toolbox.population(n=100)
cxpb = P_CROSSOVER
mutpb = P_MUTATION
ngen=MAX_GENERATIONS
# Évaluation de la population initiale
fitnesses = list(map(toolbox.evaluate, population))
for ind, fit in zip(population, fitnesses):
    ind.fitness.values = fit

# Affichage de la meilleure solution de la population initiale
best_ind = tools.selBest(population, k=1)[0]
print("Best solution : {}".format(best_ind))
print("Fitness of the best solution : {}".format(best_ind.fitness.values[0]))

    
# Lancement de l'algorithme génétique
for generation in range(50):
    print("Generation : {}".format(generation))
    
    # Sélection des parents
    parents = toolbox.select(population, len(population))
    
    # Clonage des parents
    offspring = list(map(toolbox.clone, parents))
    
    # Croisement des parents clonés
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < cxpb:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values
            
    # Mutation des enfants
    for mutant in offspring:
        if random.random() < mutpb:
            toolbox.mutate(mutant)
            del mutant.fitness.values
    
    # Évaluation des enfants avec une fitness invalide
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
        
    # Remplacement de la population par les enfants
    population[:] = offspring
    
    
    # Affichage de la meilleure solution de la génération actuelle
    best_ind = tools.selBest(population, k=1)[0]
    print("Best solution for generation : {}".format(best_ind))
    print("Fitness of the best solution : {}".format(best_ind.fitness.values[0]))
    
        
    # Génération donnée
    #generation = random.sample(range(0, len(scrubbers)), 5)
    #print("Generation:", generation)


    selected_column = [scrubber[2] for scrubber in scrubbers if scrubbers.index(scrubber) in best_ind]
    selected_time = [scrubber[1] for scrubber in scrubbers if scrubbers.index(scrubber) in best_ind]
    # Addition des valeurs sélectionnées
    total = sum(selected_column)
    time = max(selected_time)
    # Affichage du total
    print("Total area cleaned:", total,"m²")
    print("Total area time:", time ,"hours")
    
print("-- End of the evolution --")

best_ind = tools.selBest(population, k=1)[0]
print("Best solution Found : {}".format(best_ind))
print("Fitness of the best solution : {}".format(best_ind.fitness.values[0]))




D:\Programme\Anaconda\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
D:\Programme\Anaconda\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	min                                     
0  	400   	((0.0, 290994000.0), (0.0, 404322000.0))
1  	400   	((0.0, 245052000.0), (0.0, 183740000.0))
2  	400   	((0.0, 150335000.0), (0.0, 286703000.0))
3  	400   	((0.0, 149990000.0), (0.0, 170445000.0))
4  	400   	((0.0, 118125000.0), (0.0, 159054000.0))
5  	400   	((0.0, 133974000.0), (0.0, 134460000.0))
6  	400   	((0.0, 124913000.0), (0.0, 118125000.0))
7  	400   	((0.0, 120509000.0), (0.0, 115290000.0))
8  	400   	((0.0, 112455000.0), (0.0, 115745000.0))
9  	400   	((0.0, 106715000.0), (0.0, 106715000.0))
10 	400   	((0.0, 103880000.0), (0.0, 103880000.0))
11 	400   	((0.0, 103880000.0), (0.0, 103880000.0))
12 	400   	((0.0, 103880000.0), (0.0, 103880000.0))
13 	400   	((0.0, 103880000.0), (0.0, 103880000.0))
14 	400   	((0.0, 103880000.0), (0.0, 103880000.0))
15 	400   	((0.0, 103880000.0), (0.0, 103880000.0))
16 	400   	((0.0, 103880000.0), (0.0, 103880000.0))
17 	400   	((0.0, 103880000.0), (0.0, 103880000.0))
18 	400   	(

166	400   	((0.0, 103880000.0), (0.0, 103880000.0))
167	400   	((0.0, 103880000.0), (0.0, 103880000.0))
168	400   	((0.0, 103880000.0), (0.0, 103880000.0))
169	400   	((0.0, 103880000.0), (0.0, 103880000.0))
170	400   	((0.0, 103880000.0), (0.0, 103880000.0))
171	400   	((0.0, 103880000.0), (0.0, 103880000.0))
172	400   	((0.0, 103880000.0), (0.0, 103880000.0))
173	400   	((0.0, 103880000.0), (0.0, 103880000.0))
174	400   	((0.0, 103880000.0), (0.0, 103880000.0))
175	400   	((0.0, 103880000.0), (0.0, 103880000.0))
176	400   	((0.0, 103880000.0), (0.0, 103880000.0))
177	400   	((0.0, 103880000.0), (0.0, 103880000.0))
178	400   	((0.0, 103880000.0), (0.0, 103880000.0))
179	400   	((0.0, 103880000.0), (0.0, 103880000.0))
180	400   	((0.0, 103880000.0), (0.0, 103880000.0))
181	400   	((0.0, 103880000.0), (0.0, 103880000.0))
182	400   	((0.0, 103880000.0), (0.0, 103880000.0))
183	400   	((0.0, 103880000.0), (0.0, 103880000.0))
184	400   	((0.0, 103880000.0), (0.0, 103880000.0))
185	400   	(

Best solution for generation : [0, 0, 0, 0, 0]
Fitness of the best solution : 0.0
Total area cleaned: 1225 m²
Total area time: 1 hours
Generation : 38
Best solution for generation : [0, 0, 0, 0, 0]
Fitness of the best solution : 0.0
Total area cleaned: 1225 m²
Total area time: 1 hours
Generation : 39
Best solution for generation : [0, 0, 0, 0, 0]
Fitness of the best solution : 0.0
Total area cleaned: 1225 m²
Total area time: 1 hours
Generation : 40
Best solution for generation : [0, 0, 0, 0, 0]
Fitness of the best solution : 0.0
Total area cleaned: 1225 m²
Total area time: 1 hours
Generation : 41
Best solution for generation : [0, 0, 0, 0, 0]
Fitness of the best solution : 0.0
Total area cleaned: 1225 m²
Total area time: 1 hours
Generation : 42
Best solution for generation : [0, 0, 0, 0, 0]
Fitness of the best solution : 0.0
Total area cleaned: 1225 m²
Total area time: 1 hours
Generation : 43
Best solution for generation : [0, 0, 0, 0, 0]
Fitness of the best solution : 0.0
Total area c

In [67]:
def calculate_total_time(individual):
    total_time = 0
    for site, scrubber_idx in zip(sites, individual):
        scrubber = scrubbers[scrubber_idx]
        total_time += site[2] / scrubber[1]  # Fix the calculation here
    return total_time
# Génération donnée
generation = random.sample(range(0, len(scrubbers)), 5)
print("Generation:", generation)


selected_column = [scrubber[2] for scrubber in scrubbers if scrubbers.index(scrubber) in generation]

# Addition des valeurs sélectionnées
total = sum(selected_column)

# Affichage du total
print("Total area cleaned:", total,"m²")
total_time_used = calculate_total_time(generation)
print("Total time used: {} hours".format(total_time_used))

Generation: [12, 18, 15, 9, 16]
Total area cleaned: 21855 m²
Total time used: 6.138888888888889 hours
